# Modelling herb runs in Old School Runescape

## 03-Data collection & modelling
As the only way to interface with Old School Runescapes Farming skill is through playing the game, my method of data collection will be to record data while doing herb runs myself. I will use a .csv formatted by the rows denoting the herb run's number and the columns denoting the herb patches location. After a successful harvest at a particular patch I will enter the exact number of herbs picked into the corresponding entry by herb patch location and herb run number. Note that the number I enter in each entry will also include the minimum possible amount of herbs harvested based on the compost I used on the herb patch. In the case of ultra-compost I am guaranteed to pick 6 herbs. So in order to make correct statistical inference on the probability in question I will need to subtract 6 from each entry when doing analysis. In order to lock down as many parameters related to the calculation of the first failure to harvest an extra herb from a patch, I will plant the same type of herb in each of the patches. The herb I will plant is the highest tier herb available, torstol. As for the other dependencies of the probability in question I will remain at 85 farming over the data collection period, I will always treat each herb patch with ultra-compost and I will always be using magic secateurs while harvesting. This ensures that each herb run will be an independent sample as the other known parameters will remain fixed over each harvest between particular patches. Also as each individual patch will have a different probability of first failure to harvest a herb we cannot combine the data for each herb patch, thus, the need to seperate the data by herb patch location. So we are going to be computing eight instances of this probability, one for each herb patch. Hence, we are really completing eight instances of the MLE detailed previously.

The eight herb patches will be labelled by their in-game location as: Farming Guild (FG), Port Phasmatys (PP), Weiss (W), Catherby (C), Troll Stronghold (TS), Hosidius (H), Ardougne (A) and Falador (F). So the probability distribution that we will be using MLE to estmate for each patch will be (where $X_k$ for $k$ being one of the above names, will be the amount of extra herbs harvested on a single herb patch and $p_k$ will be the true probability of successfully picking an extra herb from a single herb patch)
$$P(X_k = x_k) = p_k^{x_k}(1-p_k),$$

where MLE gave our estimate of $p_k$ to be
$$\text{MLE}(p_k) = \frac{\bar{X_k}}{1+\bar{X_k}}.$$

Now as for the probability distribution of the extra amount of herbs harvested on a single herb run, we can use the fact that each herb patch is independent of one another to construct the joint probability distribution as the product of the above individual distributions. i.e. if we consider a realisation of a the amount of extra herbs harvested on a herb run to be an eight-dimensional vector where each coordinate is a realisation of the amount of extra herbs harvested on a particular patch labelled by the same $k$ as above then
$$P\left(X_{\text{FG}} = x_{\text{FG}},X_{\text{PP}} = x_{\text{PP}}\dots,X_{\text{F}} = x_{\text{F}}\right) = \prod_{k}p_k^{x_k}(1-p_k).$$

Then the probability of harvesting $t$ extra herbs on a single herb run will be very similar to the above distribution, just with the different cases where the sum of all random variables equal a particular $t$ being added together. It is important to note the difference between the above joint distribution and the probability distribution of the sum of each $X_k$. In particular when I begin my later exploratory analysis on the gathered data I will mention these two distributions seperately. 

Now we can consider calculating the probabilty distribution of the sum of each $X_k$. As each $X_k$ is independent but not identically distributed $NB(1,p_k)$, we can consider using the properties of moment generating functions of probability distributions to find the probability distribution of this sum. As can be found [here](https://en.wikipedia.org/wiki/Negative_binomial_distribution) the moment generating function of a negative binomial random variable is known. So we can apply a particular result on sums of independent random variables which states that the moment generating function of the sum of random variables is the product of the individual moment generating functions (see [here](https://en.wikipedia.org/wiki/Moment-generating_function#Linear_combination_of_independent_random_variables)). i.e. if we let $T$ be the random variable such that
$$T = \sum_{k}X_k, $$

with moment generating function $M_{T}(s)$. Then the result states that 
$$M_{T}(s) = \prod_{k}M_{X_k}(s).$$

The moment generating function of a $NB(1,p_k)$ distributed random variable is (for appropriate $s$)
$$M_{X_k}(s) = \frac{1-p_k}{1-p_ke^{s}}.$$

Hence, by the previously noted result
$$M_{T}(s) = \prod_{k}\frac{1-p_k}{1-p_ke^{s}}.$$

Now to find the probability distribution from this moment generating function we can consider the exact definition of the moment generating function and recall the value of the infinite geometric series (knowing that $0 < p_ke^s < 1$)
$$\begin{aligned}
M_{T}(s) &= \prod_{k}\frac{1-p_k}{1-p_ke^{s}} \\
\sum_{t=0}^{\infty} P(T = t)e^{st} &= \prod_{k}\sum_{t_k = 0}^{\infty}(1-p_k)p^{t_k}_ke^{st_k}.
\end{aligned}$$

Now in this case we must be very careful when dealing with finite products of infinite series. We can use the Cauchy product given that we know that each series is absolutely convergent (see more [here](https://en.wikipedia.org/wiki/Cauchy_product#Products_of_finitely_many_infinite_series))
$$
\sum_{t=0}^{\infty} P(T = t)e^{st} = \sum_{t_{\text{FG}} = 0}^\infty \sum_{t_{\text{PP}} = 0}^{t_{\text{FG}}}\dots\sum_{t_{\text{F}} = 0}^{t_{\text{A}}}a(\text{FG},t_{\text{F}})a(\text{PP},t_{\text{A}} - t_{\text{F}})\dots a(\text{F},t_{\text{FG}} - t_{\text{PP}}),
$$

where each 
$$a(k,u) = (1-p_{k})p_k^{u}e^{su}.$$

We can consider pulling out each exponential term which leaves each $a$ term as the PMF of each individual patch
$$\sum_{t=0}^{\infty} P(T = t)e^{st} = \sum_{t_{\text{FG}} = 0}^\infty \sum_{t_{\text{PP}} = 0}^{t_{\text{FG}}}\dots\sum_{t_{\text{F}} = 0}^{t_{\text{A}}}\left[P(X_{\text{FG}} = t_{\text{F}})P(X_{\text{PP}} = t_{\text{A}} - t_{\text{F}})\dots P(X_{\text{F}} = t_{\text{FG}} - t_{\text{PP}})\right]e^{s(t_{\text{F}} + t_{\text{A}} - t_{\text{F}} + \dots + t_{\text{FG}} - t_{\text{PP}})}.$$

So we can see that the sum present in the exponential term telescopes down to $t_{\text{FG}}$ which leaves
$$ \sum_{t=0}^{\infty} P(T = t)e^{st} = \sum_{t_{\text{FG}} = 0}^\infty \sum_{t_{\text{PP}} = 0}^{t_{\text{FG}}}\dots\sum_{t_{\text{F}} = 0}^{t_{\text{A}}}\left[P(X_{\text{FG}} = t_{\text{F}})P(X_{\text{PP}} = t_{\text{A}} - t_{\text{F}})\dots P(X_{\text{F}} = t_{\text{FG}} - t_{\text{PP}})\right]e^{s(t_{\text{FG}})}$$

Thus, equating both infinite series term by term (i.e. equating $t$ on the left with $t_{\text{FG}}$ on the right) gives the probability distribution in question to be
$$P(T = t) = \sum_{t_{\text{PP}} = 0}^{t}\dots\sum_{t_{\text{F}} = 0}^{t_{\text{A}}}P(X_{\text{FG}} = t_{\text{F}})P(X_{\text{PP}} = t_{\text{A}} - t_{\text{F}})\dots P(X_{\text{F}} = t - t_{\text{PP}}).$$

We can note that the form of the above probability distribution of the sum is exactly as we thought it would be, where the probabilities of reaching the same number of herbs in the joint probability distribution are summed together to construct the sum distribution. So an alternative formulation of the above convolution is
$$P(T = t) = \sum_{s\in\mathcal{S}(t)} \prod_{k}P(X_k = s_k), \ \text{where} \ \mathcal{S}(t) = \left\{s = (s_{\text{FG}},s_{PP},\dots,s_{\text{F}}): s_{\text{FG}} + s_{PP}+ \dots +s_{\text{F}} = t\right\}.$$

This set $\mathcal{S}(t)$ is the set of all positive integer partitions (including zero) of $t$ of fixed length $8$, the number of herb patches. It is very important to note that for large $t$ this set is extremely large. Thus, as $t$ gets large this finite sum will be computationally difficult to solve. 

